In [82]:
import collections
import math
import re
import sys
import os
sys.path.append(os.path.abspath("../"))
import itertools
from matplotlib import pyplot as plt
import functools
import pyparsing
import pandas as pd
import numpy as np

In [2]:
from utils import *
from aoc import *
from personal import SESSION

In [3]:
aoc = AOC(session=SESSION)

In [4]:
aoc.verify_session()

True

In [30]:
data = aoc.get_today_file().analyse().head().data

Local file found.
143 empty line(s) found. Analyse as multiline data.
===== HEAD (5) =====
['Tile 2903:', '..#..#....', '.....#...#', '..#...#.##', '#....#..#.', '#.....#..#', '........##', '........#.', '#..##.....', '##.......#', '##.....##.']
['Tile 3659:', '###....##.', '....#...#.', '..........', '......#...', '#.........', '###.#.#...', '#....#...#', '#..#..#...', '....#.....', '..#.##..##']
['Tile 3691:', '#..##...#.', '.........#', '##.#.....#', '.#........', '.......#..', '..........', '##...#....', '..........', '#...#..#..', '###.#.#.##']
['Tile 3391:', '.#####.##.', '#.....#.##', '####......', '...#.#..##', '.#.#.#....', '#.#......#', '#.......##', '.#...#...#', '#.##.....#', '...#..#.#.']
['Tile 1867:', '.##....#.#', '........#.', '#...#....#', '..........', '.........#', '.........#', '#.#.#.....', '#...#.#...', '....#..#.#', '.##.##.###']


In [223]:
piece_size = len(data[0][1:])

In [63]:
def get_borders(a):
    a_borders = [a[0], a[-1], ''.join([v[0] for v in a]), ''.join([v[-1] for v in a])]
    for poss in a_borders.copy():
        a_borders.append(poss[::-1])
    return a_borders

In [72]:
def compare(a, b):
    ba = get_borders(a)
    bb = get_borders(b)
    # print(ba, bb)
    return bool(set(ba).intersection(bb))

In [238]:
matrice = collections.defaultdict(dict)
data_up = dict()
for d1 in data:
    t1, d1 = d1[0], d1[1:]
    t1 = t1.replace('Tile ', '').replace(':', '')
    data_up[t1] = d1
    for d2 in data:
        t2, d2 = d2[0], d2[1:]
        t2 = t2.replace('Tile ', '').replace(':', '')
        if t1 == t2:
            matrice[t1][t2] = False
        else:
            matrice[t1][t2] = compare(d1, d2)
        matrice[t2][t1] = matrice[t1][t2]

In [136]:
df = pd.DataFrame(matrice)
# df.replace(False, np.NaN)
df.head()

,2903,3659,3691,3391,1867,3259,2441,2477,3533,3877,...,3919,1237,2803,1559,3083,1549,2777,1621,1847,1759
2903,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3659,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3691,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3391,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1867,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [137]:
res = collections.defaultdict(list)
for column in df:
    res[df[column].value_counts()[True]].append(column)

In [138]:
math.prod(map(int, res[2]))

19955159604613

## P2

In [139]:
def get_adj(id_):
    return tuple(df[str(id_)][df[str(id_)]].keys())

In [140]:
res[2]

['1867', '2441', '2633', '1663']

In [143]:
size = int(len(data)**0.5)

In [200]:
def common_adj(a, b, fmap):
    return set(get_adj(a)).intersection(get_adj(b)) - set([v for l in fmap for v in l])

In [207]:
def get_inti_adj(a, fmap):
    return set(get_adj(a)) - set([v for l in fmap for v in l])

In [215]:
final_map = [[0]*size for i in range(size)]

In [216]:
final_map[0][0] = res[2][0]
final_map[1][0], final_map[0][1] = get_adj(final_map[0][0])
for i in range(3, size*2):
    j = i%2
    k = i//2
    print(j, k)
    if j == 1:
        # print(final_map[j][k-1], final_map[j-1][k])
        # print(common_adj(final_map[j][k-1], final_map[j-1][k], final_map))
        final_map[j][k] = list(common_adj(final_map[j][k-1], final_map[j-1][k], final_map))[0]
    else:
        final_map[j][k] = list(common_adj(final_map[j][k-1], final_map[j][k-1], final_map))[0]



1 1
0 2
1 2
0 3
1 3
0 4
1 4
0 5
1 5
0 6
1 6
0 7
1 7
0 8
1 8
0 9
1 9
0 10
1 10
0 11
1 11


In [217]:
for i in range(1, size-1):
    for j in range(0, size):
        final_map[i+1][j] = list(get_inti_adj(final_map[i][j], final_map))[0]

In [250]:
maps = pd.DataFrame(final_map).applymap(lambda x: data_up[x])

In [302]:
def rotate(el, i, reverse=False):
    if reverse:
        el = np.flip(el, 0)
    if i <= 0:
        return np.flip(el, 0)
    if i == 1:
        return el
    if i == 2:
        return np.flip(el, 1).T
    if i == 3:
        return np.flip(el, 0).T
    return rotate(np.flip(el, 1), i-4)

In [304]:
rotate(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), 0, reverse=True)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [305]:
def find_rotate(el, bottom):
    elb = get_borders(el)
    bob = get_borders(bottom)
    for ai, a in enumerate(elb):
        for bi, b in enumerate(bob):
            if a == b:
                return ai, bi

In [309]:
def set_portion(m, sm, i, j, r, reverse=False):
    sm = rotate([list(el) for el in sm], r, reverse)
    for k, l in enumerate(sm):
        for f, v in enumerate(l):
            m[i+k][j+f] = v

In [320]:
cmap = [['?']*piece_size*size for i in range(piece_size*size)]

In [325]:
set_portion(cmap, maps[0][0], 0, 0, 0)
for i in range(0, size-1):
    r1, r2 = find_rotate(maps[0][i], maps[0][i+1])
    set_portion(cmap, maps[0][i+1], piece_size+1, 0, r2)

In [326]:
for l in cmap:
    print(''.join(l))

.##.##.###??????????????????????????????????????????????????????????????????????????????????????????????????????????????
....#..#.#??????????????????????????????????????????????????????????????????????????????????????????????????????????????
#...#.#...??????????????????????????????????????????????????????????????????????????????????????????????????????????????
#.#.#.....??????????????????????????????????????????????????????????????????????????????????????????????????????????????
.........#??????????????????????????????????????????????????????????????????????????????????????????????????????????????
.........#??????????????????????????????????????????????????????????????????????????????????????????????????????????????
..........??????????????????????????????????????????????????????????????????????????????????????????????????????????????
#...#....#??????????????????????????????????????????????????????????????????????????????????????????????????????????????
........#.??????????????????????